In [1]:
import pandas as pd

In [2]:
# CSVs laden
personen_df = pd.read_csv("../data/personen.csv")
teilaufgaben_df = pd.read_csv("../data/teilaufgaben.csv")
projekte_df = pd.read_csv("../data/projekte.csv")

In [3]:
# Kompetenzspalte vorbereiten
personen_df["kompetenzen_liste"] = personen_df["kompetenzen"].str.split(", ")

In [4]:
# Belegungstracker: person_id → monat (z. B. "03/2025") → belegte Kapazität
belegungen = {}

In [4]:
# Matching-Funktion
matching_ergebnisse = []

for _, ta in teilaufgaben_df.iterrows():
    ta_start = int(ta["start"].split("/")[0])
    ta_ende = int(ta["ende"].split("/")[0])
    kandidaten = []

    for _, person in personen_df.iterrows():
        if ta["kompetenz"] not in person["kompetenzen_liste"]:
            continue

        # Verfügbarkeit über Zeitfenster aufsummieren
        verfuegbarkeit_summe = 0
        for monat in range(ta_start, ta_ende + 1):
            col = f"verfuegbarkeit_{monat:02d}/2025"
            if col in personen_df.columns:
                verfuegbarkeit_summe += person[col]
                
        score = round(verfuegbarkeit_summe * person["zeitbudget"], 2)

        kandidaten.append({
            "projekt_id": ta["projekt_id"],
            "teilaufgabe_id": ta["teilaufgabe_id"],
            "teilaufgabe": ta["bezeichnung"],
            "kompetenz": ta["kompetenz"],
            "person_id": person["id"],
            "name": person["name"],
            "score": score,
            "verfuegbarkeit_summe": round(verfuegbarkeit_summe, 2)
        })

    top_kandidaten = sorted(kandidaten, key=lambda x: x["score"], reverse=True)[:3]
    matching_ergebnisse.extend(top_kandidaten)


In [5]:
# Ergebnisliste
matching_ergebnisse = []

for _, ta in teilaufgaben_df.iterrows():
    ta_start = int(ta["start"].split("/")[0])
    ta_ende = int(ta["ende"].split("/")[0])
    ta_monate = ta_ende - ta_start + 1
    ta_monatsaufwand = ta["aufwand"] / ta_monate  # gleichmäßige Verteilung

    kandidaten = []

    for _, person in personen_df.iterrows():
        if ta["kompetenz"] not in person["kompetenzen_liste"]:
            continue

        person_id = person["id"]
        ausreichend_frei = True
        verfügbarkeit_summe = 0

        for monat in range(ta_start, ta_ende + 1):
            monat_label = f"{monat:02d}/2025"
            verfügbarkeit = person.get(f"verfuegbarkeit_{monat_label}", 0)
            verbraucht = belegungen.get(person_id, {}).get(monat_label, 0)
            rest = verfügbarkeit - verbraucht

            if rest < ta_monatsaufwand:
                ausreichend_frei = False
                break
            verfügbarkeit_summe += rest

        if ausreichend_frei:
            score = round(verfügbarkeit_summe * person["zeitbudget"], 2)
            kandidaten.append({
                "projekt_id": ta["projekt_id"],
                "teilaufgabe_id": ta["teilaufgabe_id"],
                "teilaufgabe": ta["bezeichnung"],
                "kompetenz": ta["kompetenz"],
                "person_id": person_id,
                "name": person["name"],
                "score": score,
                "verfuegbarkeit_summe": round(verfügbarkeit_summe, 2)
            })

    # Beste Kandidaten wählen
    top_kandidaten = sorted(kandidaten, key=lambda x: x["score"], reverse=True)[:3]

    # Belegung eintragen
    for kandidat in top_kandidaten:
        pid = kandidat["person_id"]
        if pid not in belegungen:
            belegungen[pid] = {}
        for monat in range(ta_start, ta_ende + 1):
            monat_label = f"{monat:02d}/2025"
            belegungen[pid][monat_label] = belegungen[pid].get(monat_label, 0) + ta_monatsaufwand

    matching_ergebnisse.extend(top_kandidaten)


In [6]:
# Ergebnis speichern
pd.DataFrame(matching_ergebnisse).to_csv("matching_ergebnis.csv", index=False)
print("Matching abgeschlossen – Ergebnisse gespeichert in 'matching_ergebnis.csv'")

Matching abgeschlossen – Ergebnisse gespeichert in 'matching_ergebnis.csv'
